In [ ]:
!pip install torchvision torchaudio numpy pandas spacy datasets sacrebleu matplotlib tqdm
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 144.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install torch==2.3.1 torchtext==0.18.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 102.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

KeyboardInterrupt: 

In [ ]:
class Config:
    def __init__(self):
        # Thiết lập dữ liệu [cite: 4, 5]
        self.src_lang = 'vi'
        self.tgt_lang = 'en'
        self.max_len = 100         # Độ dài tối đa của câu (để cắt/pad)
        self.batch_size = 32

        # Thiết lập Mô hình (Model Architecture) [cite: 10]
        self.d_model = 512        # Kích thước vector embedding
        self.n_heads = 8          # Số lượng Head trong Multi-Head Attention
        self.n_layers = 6         # Số lớp Encoder và Decoder
        self.d_ff = 2048          # Kích thước lớp ẩn trong Feed Forward Network
        self.dropout = 0.1

        # Thiết lập Huấn luyện (Training) [cite: 27, 29]
        self.lr = 0.0001          # Learning rate
        self.epochs = 20
        self.warmup_steps = 4000  # Cho Scheduler
        self.label_smoothing = 0.1 # Kỹ thuật giúp model đỡ overfit

        # Đường dẫn lưu model
        self.model_path = 'weights/transformer_vi_en.pth'

cfg = Config()

In [ ]:
from datasets import load_dataset
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.data.utils import get_tokenizer
from collections import Counter
import matplotlib.pyplot as plt
import torch.nn as nn
import sacrebleu
import math
import torch.optim as optim
import time
import json
import os


# Thiết lập các token đặc biệt (Rất quan trọng!)
PAD_TOKEN = '<pad>' # Dùng để đệm câu cho bằng độ dài
SOS_TOKEN = '<sos>' # Start of Sentence - Báo hiệu bắt đầu câu
EOS_TOKEN = '<eos>' # End of Sentence - Báo hiệu kết thúc câu
UNK_TOKEN = '<unk>' # Unknown - Dùng cho các từ không có trong từ điển

PAD_IDX = 0
SOS_IDX = 1
EOS_IDX = 2
UNK_IDX = 3

In [ ]:
class Vocabulary:
    def __init__(self, freq_threshold=2):
        self.itos = {0: "<pad>", 1: "<sos>", 2: "<eos>", 3: "<unk>"}
        self.stoi = {"<pad>": 0, "<sos>": 1, "<eos>": 2, "<unk>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_en(text):
        # Sử dụng tokenizer chuẩn cho tiếng Anh
        tokenizer = get_tokenizer('basic_english')
        return tokenizer(text)

    @staticmethod
    def tokenizer_vi(text):
        # CẢI TIẾN: Hàm tách từ tiếng Việt
        # Nếu cài được thư viện 'pyvi' hoặc 'underthesea' thì thay vào đây
        # Tạm thời dùng split() nhưng xử lý kỹ hơn về dấu câu
        text = text.lower().strip()
        # Tách dấu câu đơn giản (để dấu chấm, phẩy không dính vào từ)
        for char in ['.', ',', '?', '!', '"', "'"]:
            text = text.replace(char, f" {char} ")
        return text.split()

    def build_vocabulary(self, sentence_list, lang='en'):
        frequencies = Counter()
        idx = 4

        tokenizer_fn = self.tokenizer_en if lang == 'en' else self.tokenizer_vi

        for sentence in sentence_list:
            for word in tokenizer_fn(sentence):
                frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text, lang='en'):
        tokenizer_fn = self.tokenizer_en if lang == 'en' else self.tokenizer_vi
        tokenized_text = tokenizer_fn(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<unk>"]
            for token in tokenized_text
        ]

    # Thêm chức năng lưu/tải từ điển để không phải build lại
    def save_vocab(self, path):
        with open(path, 'w', encoding='utf-8') as f:
            json.dump(self.stoi, f, ensure_ascii=False)

    def load_vocab(self, path):
        with open(path, 'r', encoding='utf-8') as f:
            self.stoi = json.load(f)
            self.itos = {v: k for k, v in self.stoi.items()}

In [ ]:
class BilingualDataset(Dataset):
    def __init__(self, src_sentences, tgt_sentences, src_vocab, tgt_vocab):
        self.src_sentences = src_sentences
        self.tgt_sentences = tgt_sentences
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    def __len__(self):
        return len(self.src_sentences)

    def __getitem__(self, index):
        src_text = self.src_sentences[index]
        tgt_text = self.tgt_sentences[index]

        # Map từ -> số
        src_indices = [self.src_vocab.stoi["<sos>"]] + \
                      self.src_vocab.numericalize(src_text, lang='vi') + \
                      [self.src_vocab.stoi["<eos>"]]

        tgt_indices = [self.tgt_vocab.stoi["<sos>"]] + \
                      self.tgt_vocab.numericalize(tgt_text, lang='en') + \
                      [self.tgt_vocab.stoi["<eos>"]]

        return torch.tensor(src_indices), torch.tensor(tgt_indices)

In [ ]:
class Collate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        src_batch, tgt_batch = [], []
        for src_item, tgt_item in batch:
            src_batch.append(src_item)
            tgt_batch.append(tgt_item)

        # Padding
        src_batch = pad_sequence(src_batch, padding_value=self.pad_idx, batch_first=True)
        tgt_batch = pad_sequence(tgt_batch, padding_value=self.pad_idx, batch_first=True)

        return src_batch, tgt_batch

In [ ]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model):
        super(TokenEmbedding, self).__init__()
        self.d_model = d_model
        # Lớp Embedding chuẩn của PyTorch
        self.emb = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # Theo paper gốc "Attention Is All You Need", ta nhân embedding với căn bậc 2 của d_model
        # Lý do: Để giá trị của embedding có độ lớn tương đương với Positional Encoding sắp cộng vào
        return self.emb(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000, dropout=0.1):
        """
        Args:
            d_model: Kích thước vector embedding (thường là 512)
            max_len: Độ dài tối đa của câu mà mô hình hỗ trợ
            dropout: Xác suất dropout để tránh overfitting
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Tạo ma trận PE kích thước (max_len, d_model) chứa toàn số 0
        pe = torch.zeros(max_len, d_model)

        # Tạo vector vị trí (pos): [0, 1, 2, ..., max_len-1]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # Tính div_term (mẫu số trong công thức): 10000^(2i/d_model)
        # Sử dụng log space để tính toán ổn định hơn về mặt số học
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # Áp dụng công thức Sin cho vị trí chẵn (2i)
        pe[:, 0::2] = torch.sin(position * div_term)

        # Áp dụng công thức Cos cho vị trí lẻ (2i+1)
        pe[:, 1::2] = torch.cos(position * div_term)

        # Thêm 1 chiều batch ở đầu: (1, max_len, d_model) để dễ cộng với input
        pe = pe.unsqueeze(0)

        # register_buffer giúp lưu trữ tensor này vào state_dict của mô hình
        # nhưng không cập nhật nó trong quá trình backpropagation (vì nó cố định)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Input tensor kích thước (batch_size, seq_len, d_model)
        """
        # Cắt ma trận PE cho khớp với độ dài câu hiện tại (seq_len)
        # x.size(1) chính là độ dài câu thực tế
        x = x + self.pe[:, :x.size(1)]

        return self.dropout(x)

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        """
        Args:
            d_model: Kích thước vector embedding (ví dụ: 512)
            n_heads: Số lượng 'đầu' chú ý (ví dụ: 8)
        """
        super(MultiHeadAttention, self).__init__()

        assert d_model % n_heads == 0, "d_model phải chia hết cho n_heads"

        self.d_head = d_model // n_heads
        self.n_heads = n_heads
        self.d_model = d_model

        # 1. Các lớp Linear để tạo ra Q, K, V từ đầu vào
        # W_q, W_k, W_v trong lý thuyết
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        # Lớp Linear cuối cùng sau khi nối các heads lại
        self.fc_out = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask=None):
        """
        Args:
            query: (Batch, Seq_len, D_model)
            key:   (Batch, Seq_len, D_model)
            value: (Batch, Seq_len, D_model)
            mask:  Tensor chứa giá trị 0 hoặc 1 để che đi các vị trí không cần thiết
        """
        batch_size = query.shape[0]

        # 1. Tính toán Q, K, V
        Q = self.w_q(query) # (Batch, Seq_len, D_model)
        K = self.w_k(key)   # (Batch, Seq_len, D_model)
        V = self.w_v(value) # (Batch, Seq_len, D_model)

        # 2. Chia nhỏ thành n_heads
        # Biến đổi: (Batch, Seq_len, D_model) -> (Batch, Seq_len, n_heads, d_head)
        # Sau đó đảo trục để n_heads lên trước: (Batch, n_heads, Seq_len, d_head)
        # Việc này giúp tính toán song song các heads
        Q = Q.view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)
        K = K.view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)
        V = V.view(batch_size, -1, self.n_heads, self.d_head).transpose(1, 2)

        # 3. Scaled Dot-Product Attention
        # Tính điểm năng lượng: (Batch, n_heads, Seq_len_Q, Seq_len_K)
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_head)

        # 4. Áp dụng Mask (Nếu có)
        # Mask thường dùng để:
        # - Che padding (Padding Mask)
        # - Che các từ tương lai trong Decoder (Look-ahead Mask)
        if mask is not None:
            # Những chỗ mask == 0 sẽ bị gán giá trị âm vô cùng (-1e9)
            # Để khi qua Softmax nó sẽ bằng 0
            scores = scores.masked_fill(mask == 0, -1e9)

        # 5. Softmax để ra xác suất chú ý
        attention_weights = torch.softmax(scores, dim=-1)

        # 6. Nhân với V
        out = torch.matmul(attention_weights, V) # (Batch, n_heads, Seq_len_Q, d_head)

        # 7. Ghép lại (Concatenate)
        # Đảo trục lại: (Batch, Seq_len_Q, n_heads, d_head)
        out = out.transpose(1, 2).contiguous()

        # Gom lại thành vector d_model ban đầu: (Batch, Seq_len_Q, D_model)
        out = out.view(batch_size, -1, self.d_model)

        # 8. Lớp Linear cuối cùng
        out = self.fc_out(out)

        return out

In [ ]:
class PositionwiseFeedForward(nn.Module):
    """
    Mạng Feed-Forward (FFN) được áp dụng cho từng vị trí riêng biệt và giống hệt nhau.
    Công thức: FFN(x) = max(0, xW1 + b1)W2 + b2
    """
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        # Mở rộng kích thước từ d_model lên d_ff (thường gấp 4 lần, ví dụ 512 -> 2048)
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
        self.relu = nn.ReLU()

    def forward(self, x):
        # x -> Linear -> ReLU -> Dropout -> Linear
        return self.w_2(self.dropout(self.relu(self.w_1(x))))

In [ ]:
class EncoderLayer(nn.Module):
    """
    Một lớp Encoder bao gồm 2 phần chính:
    1. Multi-Head Self-Attention
    2. Position-wise Feed-Forward Network
    Mỗi phần đều có Residual Connection (Add) và Layer Normalization (Norm).
    """
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()

        # [cite_start]Thành phần 1: Self-Attention [cite: 17]
        self.self_attn = MultiHeadAttention(d_model, n_heads)
        self.norm1 = nn.LayerNorm(d_model) # [cite: 18]

        # [cite_start]Thành phần 2: Feed Forward [cite: 19]
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm2 = nn.LayerNorm(d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask):
        # 1. Sub-layer 1: Self-Attention
        # Residual connection: x + Sublayer(x)
        _src = self.self_attn(src, src, src, src_mask)
        src = self.norm1(src + self.dropout(_src))

        # 2. Sub-layer 2: Feed Forward
        _src = self.ffn(src)
        src = self.norm2(src + self.dropout(_src))

        return src

In [ ]:
class DecoderLayer(nn.Module):
    """
    Một lớp Decoder phức tạp hơn, bao gồm 3 phần:
    1. Masked Multi-Head Self-Attention (để không nhìn thấy tương lai)
    2. Multi-Head Cross-Attention (nhìn vào Encoder output)
    3. Position-wise Feed-Forward Network
    """
    def __init__(self, d_model, n_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()

        # [cite_start]1. Masked Self-Attention [cite: 21]
        self.self_attn = MultiHeadAttention(d_model, n_heads)
        self.norm1 = nn.LayerNorm(d_model)

        # [cite_start]2. Cross-Attention (Encoder-Decoder Attention) [cite: 22]
        # Query lấy từ Decoder, Key & Value lấy từ Encoder
        self.enc_dec_attn = MultiHeadAttention(d_model, n_heads)
        self.norm2 = nn.LayerNorm(d_model)

        # [cite_start]3. Feed Forward [cite: 23]
        self.ffn = PositionwiseFeedForward(d_model, d_ff, dropout)
        self.norm3 = nn.LayerNorm(d_model)

        self.dropout = nn.Dropout(dropout)

    def forward(self, tgt, enc_out, tgt_mask, src_mask):
        # Sub-layer 1: Masked Self-Attention (chú ý vào chính câu đích)
        _tgt = self.self_attn(tgt, tgt, tgt, tgt_mask)
        tgt = self.norm1(tgt + self.dropout(_tgt))

        # Sub-layer 2: Cross-Attention (chú ý vào câu nguồn - Encoder Output)
        # query=tgt, key=enc_out, value=enc_out
        _tgt = self.enc_dec_attn(tgt, enc_out, enc_out, src_mask)
        tgt = self.norm2(tgt + self.dropout(_tgt))

        # Sub-layer 3: Feed Forward
        _tgt = self.ffn(tgt)
        tgt = self.norm3(tgt + self.dropout(_tgt))

        return tgt

In [ ]:
class Encoder(nn.Module):
    def __init__(self, d_model, n_layers, n_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        # Tạo danh sách N lớp EncoderLayer
        # ModuleList giúp PyTorch quản lý các layer này
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, src, mask):
        # src đi qua từng lớp EncoderLayer lần lượt
        for layer in self.layers:
            src = layer(src, mask)
        return self.norm(src)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, d_model, n_layers, n_heads, d_ff, dropout):
        super(Decoder, self).__init__()
        # Tạo danh sách N lớp DecoderLayer
        self.layers = nn.ModuleList([
            DecoderLayer(d_model, n_heads, d_ff, dropout)
            for _ in range(n_layers)
        ])
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, tgt, memory, tgt_mask, src_mask):
        """
        tgt: Input của Decoder (câu đang dịch)
        memory: Output của Encoder (thông tin từ câu nguồn)
        """
        for layer in self.layers:
            tgt = layer(tgt, memory, tgt_mask, src_mask)
        return self.norm(tgt)

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        src_vocab_size,
        tgt_vocab_size,
        d_model=512,
        n_layers=6,
        n_heads=8,
        d_ff=2048,
        dropout=0.1,
        max_len=5000,
        src_pad_idx=0,
        tgt_pad_idx=0
    ):
        super(Transformer, self).__init__()

        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # 1. Khởi tạo phần Embedding + Positional Encoding
        self.src_embedding = TokenEmbedding(src_vocab_size, d_model)
        self.tgt_embedding = TokenEmbedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_len, dropout)

        # 2. Khởi tạo khối Encoder và Decoder
        self.encoder = Encoder(d_model, n_layers, n_heads, d_ff, dropout)
        self.decoder = Decoder(d_model, n_layers, n_heads, d_ff, dropout)

        # 3. Lớp đầu ra (Generator): Chiếu về kích thước từ điển đích
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)

        # Khởi tạo tham số (Xavier init) giúp hội tụ nhanh hơn
        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def make_src_mask(self, src):
        # Tạo mask cho Encoder: Che các vị trí padding
        # src shape: (batch_size, src_len)
        # mask shape: (batch_size, 1, 1, src_len)
        mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        return mask.to(self.device)

    def make_tgt_mask(self, tgt):
        # Tạo mask cho Decoder: Gồm 2 phần
        # 1. Padding mask: Che các vị trí padding
        # 2. Look-ahead mask: Che các từ tương lai (dạng tam giác trên)

        # Padding mask
        padding_mask = (tgt != self.tgt_pad_idx).unsqueeze(1).unsqueeze(3)

        # Look-ahead mask (tam giác)
        tgt_len = tgt.shape[1]
        look_ahead_mask = torch.tril(torch.ones(tgt_len, tgt_len)).type(torch.ByteTensor).to(self.device)

        # Kết hợp cả 2: Vừa phải không phải padding, vừa phải nằm trong quá khứ
        mask = padding_mask & look_ahead_mask

        return mask

    def forward(self, src, tgt):
        # 1. Tạo Mask
        src_mask = self.make_src_mask(src)
        tgt_mask = self.make_tgt_mask(tgt)

        # 2. Forward qua Encoder
        # Input: (Batch, Src_Len) -> Output: (Batch, Src_Len, D_Model)
        src_emb = self.positional_encoding(self.src_embedding(src))
        enc_src = self.encoder(src_emb, src_mask)

        # 3. Forward qua Decoder
        # Input: (Batch, Tgt_Len) -> Output: (Batch, Tgt_Len, D_Model)
        tgt_emb = self.positional_encoding(self.tgt_embedding(tgt))
        output = self.decoder(tgt_emb, enc_src, tgt_mask, src_mask)

        # 4. Chiếu ra xác suất từ
        return self.fc_out(output)

In [ ]:
def train_epoch(model, iterator, optimizer, criterion, clip, device):
    """
    Hàm huấn luyện cho 1 Epoch (duyệt qua toàn bộ dữ liệu 1 lần)
    """
    model.train() # Chuyển sang chế độ training (bật Dropout)
    epoch_loss = 0

    for i, (src, tgt) in enumerate(iterator):
        src = src.to(device)
        tgt = tgt.to(device)

        # Decoder Input: Bỏ token cuối cùng <eos>
        tgt_input = tgt[:, :-1]

        # Target Output: Bỏ token đầu tiên <sos> (đây là cái ta muốn model đoán)
        tgt_output = tgt[:, 1:]

        optimizer.zero_grad() # Xóa gradient cũ

        # Forward pass
        # output shape: (batch_size, tgt_len - 1, output_dim)
        output = model(src, tgt_input)

        # Reshape để tính Loss
        output_dim = output.shape[-1]
        output = output.contiguous().view(-1, output_dim)
        tgt_output = tgt_output.contiguous().view(-1)

        # Tính Loss (Cross Entropy)
        loss = criterion(output, tgt_output)

        # Backward pass
        loss.backward()

        # Cắt gradient (Gradient Clipping) để tránh bùng nổ gradient
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # Cập nhật trọng số
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion, device):
    """
    Hàm đánh giá trên tập Validation (không cập nhật trọng số)
    """
    model.eval() # Chuyển sang chế độ eval (tắt Dropout)
    epoch_loss = 0

    with torch.no_grad(): # Không tính gradient giúp chạy nhanh hơn
        for i, (src, tgt) in enumerate(iterator):
            src = src.to(device)
            tgt = tgt.to(device)

            tgt_input = tgt[:, :-1]
            tgt_output = tgt[:, 1:]

            output = model(src, tgt_input)

            output_dim = output.shape[-1]
            output = output.contiguous().view(-1, output_dim)
            tgt_output = tgt_output.contiguous().view(-1)

            loss = criterion(output, tgt_output)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def save_training_plot(train_losses, val_losses, title, filename):
    """
    Vẽ và lưu biểu đồ Loss theo Epoch
    Args:
        train_losses: List chứa giá trị loss của tập train qua từng epoch
        val_losses: List chứa giá trị loss của tập val qua từng epoch
        title: Tiêu đề biểu đồ
        filename: Tên file ảnh muốn lưu (ví dụ: 'loss_chart.png')
    """
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss', color='blue', marker='o')
    plt.plot(val_losses, label='Validation Loss', color='red', marker='x')

    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)

    # Lưu ảnh
    if not os.path.exists('reports'):
        os.makedirs('reports')
    plt.savefig(os.path.join('reports', filename))
    plt.close()
    print(f"--> Đã lưu biểu đồ tại: reports/{filename}")

def save_perplexity_plot(train_losses, val_losses, filename):
    """
    Vẽ biểu đồ Perplexity (PPL = exp(Loss))
    PPL càng thấp càng tốt.
    """
    train_ppls = [math.exp(l) for l in train_losses]
    val_ppls = [math.exp(l) for l in val_losses]

    plt.figure(figsize=(10, 6))
    plt.plot(train_ppls, label='Train PPL', color='green', linestyle='--')
    plt.plot(val_ppls, label='Val PPL', color='orange', linestyle='--')

    plt.title("Model Perplexity over Epochs")
    plt.xlabel('Epochs')
    plt.ylabel('Perplexity')
    plt.legend()
    plt.yscale('log') # Dùng thang log vì PPL có thể rất lớn lúc đầu
    plt.grid(True)

    if not os.path.exists('reports'):
        os.makedirs('reports')
    plt.savefig(os.path.join('reports', filename))
    plt.close()
    print(f"--> Đã lưu biểu đồ PPL tại: reports/{filename}")

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
def run_training():
    # 1. Cấu hình (Hyperparameters) - Nên để trong config file riêng, nhưng để đây cho tiện
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    BATCH_SIZE = 32
    N_EPOCHS = 10
    CLIP = 1
    LR = 0.0001

    # # 2. Chuẩn bị Dữ liệu (Giả lập hoặc Load thật)
    # print("--- Đang chuẩn bị dữ liệu ---")
    # # LƯU Ý: Ở đây cậu thay bằng code load file IWSLT thật
    # # Demo dữ liệu giả để code chạy được ngay
    # train_src = ["tôi là sinh viên", "máy học rất thú vị"] * 50
    # train_tgt = ["i am a student", "machine learning is interesting"] * 50
    # val_src = ["tôi đi học", "xin chào"] * 10
    # val_tgt = ["i go to school", "hello"] * 10

    print("--- Đang tải dataset IWSLT2015 (Vi-En) ---")
    dataset = load_dataset("nguyenvuhuy/iwslt2015-en-vi")

    def extract_data(data_split):
        src = [item['vi'] for item in data_split]
        tgt = [item['en'] for item in data_split]
        return src, tgt

    train_src, train_tgt = extract_data(dataset['train'])
    val_src, val_tgt = extract_data(dataset['validation'])

    # Tokenizer & Vocab
    # Removed: from utils.tokenizer import Vocabulary
    vocab_src = Vocabulary(freq_threshold=1)
    vocab_tgt = Vocabulary(freq_threshold=1)
    # Build vocab từ dữ liệu train
    vocab_src.build_vocabulary(train_src, lang='vi')
    vocab_tgt.build_vocabulary(train_tgt, lang='en')

    print(f"Vocab Source: {len(vocab_src)} | Vocab Target: {len(vocab_tgt)}")

    # DataLoader
    train_dataset = BilingualDataset(train_src, train_tgt, vocab_src, vocab_tgt)
    val_dataset = BilingualDataset(val_src, val_tgt, vocab_src, vocab_tgt)

    collate = Collate(pad_idx=0) # 0 là <pad>

    train_iterator = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)
    valid_iterator = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate)

    # 3. Khởi tạo Mô hình
    model = Transformer(
        src_vocab_size=len(vocab_src),
        tgt_vocab_size=len(vocab_tgt),
        d_model=256,    # Demo dùng nhỏ, bài thật dùng 512
        n_layers=3,     # Demo dùng 3, bài thật dùng 6
        n_heads=8,
        d_ff=512,
        dropout=0.1,
        src_pad_idx=vocab_src.stoi["<pad>"],
        tgt_pad_idx=vocab_tgt.stoi["<pad>"]
    ).to(device)

    # 4. Optimizer & Loss
    # Adam Optimizer [cite: 29]
    optimizer = optim.Adam(model.parameters(), lr=LR)

    # Cross Entropy Loss [cite: 28]
    # Quan trọng: ignore_index=0 để không tính loss cho các token <pad>
    criterion = nn.CrossEntropyLoss(ignore_index=vocab_tgt.stoi["<pad>"])

    # 5. Vòng lặp Training [cite: 30]
    best_valid_loss = float('inf')

    train_loss_history = []
    valid_loss_history = []

    print("--- Bắt đầu Training ---")
    for epoch in range(N_EPOCHS):
        start_time = time.time()

        train_loss = train_epoch(model, train_iterator, optimizer, criterion, CLIP, device)
        valid_loss = evaluate(model, valid_iterator, criterion, device)

        train_loss_history.append(train_loss)
        valid_loss_history.append(valid_loss)

        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        # Lưu model tốt nhất
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'transformer_model.pt')
            print(f"--> Đã lưu model tốt nhất tại Epoch {epoch+1}")

        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')


    print("\n--- Đang vẽ biểu đồ báo cáo ---")

    # 1. Vẽ biểu đồ Loss
    save_training_plot(
        train_loss_history,
        valid_loss_history,
        title=f'Training Loss (Epochs={N_EPOCHS})',
        filename='loss_chart.png'
    )

    # 2. Vẽ biểu đồ Perplexity
    save_perplexity_plot(
        train_loss_history,
        valid_loss_history,
        filename='perplexity_chart.png'
    )

In [ ]:
def translate_sentence(sentence, src_vocab, tgt_vocab, model, device, max_len=50):
    model.eval()

    # 1. Xử lý câu nguồn (Source)
    # Tokenize và thêm <sos>, <eos>
    # tokens = [src_vocab.stoi["<sos>"]] + src_vocab.numericalize(sentence) + [src_vocab.stoi["<eos>"]]#đã fix dòng này
    tokens = [src_vocab.stoi["<sos>"]] + src_vocab.numericalize(sentence, lang='vi') + [src_vocab.stoi["<eos>"]]
    src_tensor = torch.LongTensor(tokens).unsqueeze(0).to(device) # (1, src_len)

    # Tạo mask cho src
    src_mask = model.make_src_mask(src_tensor)

    with torch.no_grad():
        # Encode câu nguồn
        src_emb = model.positional_encoding(model.src_embedding(src_tensor))
        enc_src = model.encoder(src_emb, src_mask)

    # 2. Khởi tạo câu đích với <sos>
    tgt_indices = [tgt_vocab.stoi["<sos>"]]

    # 3. Vòng lặp Decoding
    for i in range(max_len):
        tgt_tensor = torch.LongTensor(tgt_indices).unsqueeze(0).to(device) # (1, curr_len)

        # Tạo mask cho tgt
        tgt_mask = model.make_tgt_mask(tgt_tensor)

        with torch.no_grad():
            # Decode
            tgt_emb = model.positional_encoding(model.tgt_embedding(tgt_tensor))
            output = model.decoder(tgt_emb, enc_src, tgt_mask, src_mask)

            # Lấy dự đoán cho từ cuối cùng
            pred_token_logits = model.fc_out(output[:, -1, :])

            # Chọn từ có xác suất cao nhất (Greedy)
            pred_token = pred_token_logits.argmax(1).item()

            # Nếu gặp <eos> thì dừng
            if pred_token == tgt_vocab.stoi["<eos>"]:
                break

            tgt_indices.append(pred_token)

    # 4. Chuyển từ số về chữ
    trg_tokens = [tgt_vocab.itos[i] for i in tgt_indices]

    # Bỏ <sos> ở đầu
    return trg_tokens[1:]

In [ ]:
# Hàm tính BLEU Score
def calculate_bleu(data, src_vocab, tgt_vocab, model, device):


    targets = []
    outputs = []

    for example in data:
        src = example[0] # Câu tiếng Việt gốc
        trg = example[1] # Câu tiếng Anh gốc

        prediction = translate_sentence(src, src_vocab, tgt_vocab, model, device)

        # Nối lại thành câu
        pred_sent = " ".join(prediction)

        outputs.append(pred_sent)
        targets.append([trg]) # Sacrebleu yêu cầu list of lists cho reference

    bleu = sacrebleu.corpus_bleu(outputs, targets)
    return bleu.score

In [ ]:
run_training()

--- Đang tải dataset IWSLT2015 (Vi-En) ---
Vocab Source: 21144 | Vocab Target: 47271
--- Bắt đầu Training ---


OutOfMemoryError: CUDA out of memory. Tried to allocate 2.27 GiB. GPU 

In [ ]:
def load_checkpoint_and_predict():
    # 1. Cấu hình thiết bị
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Đang sử dụng thiết bị: {device}")

    # 2. Tái tạo Vocabulary (BẮT BUỘC PHẢI KHỚP VỚI LÚC TRAIN)
    # Lưu ý: Trong dự án thực tế, bạn nên lưu vocab ra file json và load lại.
    # Ở đây mình build lại nhanh từ dữ liệu mẫu để demo.
    print("--- Đang load Vocabulary ---")
    train_src = ["tôi là sinh viên", "máy học rất thú vị"] * 50
    train_tgt = ["i am a student", "machine learning is interesting"] * 50

    # Instantiate tokenizers first
    src_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
    tgt_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

    vocab_src = Vocabulary()
    vocab_tgt = Vocabulary()
    vocab_src.build_vocabulary(train_src, lang='vi')
    vocab_tgt.build_vocabulary(train_tgt, lang='en')

    # 3. Khởi tạo lại kiến trúc mô hình (Cấu hình phải khớp với lúc train)
    print("--- Đang khởi tạo mô hình ---")
    model = Transformer(
        src_vocab_size=len(vocab_src),
        tgt_vocab_size=len(vocab_tgt),
        d_model=256,      # Khớp với train.py
        n_layers=3,       # Khớp với train.py
        n_heads=8,
        d_ff=512,
        dropout=0.1,
        src_pad_idx=vocab_src.stoi["<pad>"],
        tgt_pad_idx=vocab_tgt.stoi["<pad>"]
    ).to(device)

    # 4. Load trọng số đã train (File .pt)
    try:
        model.load_state_dict(torch.load('transformer_model.pt', map_location=device))
        print("--> Đã load trọng số từ 'transformer_model.pt' thành công!")
    except FileNotFoundError:
        print("LỖI: Không tìm thấy file 'transformer_model.pt'. Bạn đã chạy train.py chưa?")
        return

    # 5. Dịch thử
    sentences = [
        "tôi là sinh viên",
        "máy học rất thú vị",
        "tôi là sinh viên máy học" # Câu ghép thử thách hơn chút
    ]

    print("\n--- KẾT QUẢ DỊCH ---")
    for sentence in sentences:
        result = translate_sentence(
            sentence,
            vocab_src,
            vocab_tgt,
            model,
            device,
            max_len=20
        )

        print(f"Tiếng Việt: {sentence}")
        print(f"Tiếng Anh : {' '.join(result)}")
        print("-" * 30)

In [ ]:
load_checkpoint_and_predict()

Đang sử dụng thiết bị: cuda
--- Đang load Vocabulary ---
--- Đang khởi tạo mô hình ---
--> Đã load trọng số từ 'transformer_model.pt' thành công!

--- KẾT QUẢ DỊCH ---
Tiếng Việt: tôi là sinh viên
Tiếng Anh : i am am am am am
------------------------------
Tiếng Việt: máy học rất thú vị
Tiếng Anh : 
------------------------------
Tiếng Việt: tôi là sinh viên máy học
Tiếng Anh : i
------------------------------


In [ ]:
!pip install sacrebleu tqdm

In [ ]:
import sacrebleu
from tqdm import tqdm

def calculate_bleu_score(model, dataset, src_vocab, tgt_vocab, device):
    """
    Hàm tính điểm BLEU trên toàn bộ tập dữ liệu
    Args:
        dataset: List các tuple (câu nguồn, câu đích)
    """
    model.eval()

    hypotheses = [] # Danh sách các câu máy dịch (Prediction)
    references = [] # Danh sách các câu đáp án (Ground Truth)

    print(f"Đang tiến hành dịch và đánh giá {len(dataset)} câu...")

    # Dùng tqdm để hiển thị thanh tiến trình vì bước này có thể lâu
    for src_text, tgt_text in tqdm(dataset):
        # 1. Máy dịch
        pred_tokens = translate_sentence(
            src_text,
            src_vocab,
            tgt_vocab,
            model,
            device,
            max_len=50
        )

        # Nối các token lại thành câu hoàn chỉnh
        # Lưu ý: Cần join cẩn thận để tách đúng từ.
        # Ở đây ta join bằng space đơn giản cho demo.
        pred_str = " ".join(pred_tokens)

        # 2. Lưu lại kết quả
        hypotheses.append(pred_str)
        references.append(tgt_text)

    # 3. Tính điểm BLEU dùng thư viện sacrebleu
    # sacrebleu yêu cầu references phải là list of lists (vì 1 câu nguồn có thể có nhiều cách dịch)
    # Nhưng ở đây ta chỉ có 1 đáp án cho mỗi câu nên ta bọc nó lại: [references]
    bleu = sacrebleu.corpus_bleu(hypotheses, [references])

    return bleu.score

In [ ]:
    # 1. Cấu hình
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Sử dụng thiết bị: {device}")

    # 2. Tái tạo Vocabulary (PHẢI GIỐNG LÚC TRAIN)
    # Mẹo: Trong thực tế, bạn nên load vocab từ file json đã lưu.
    # Ở đây mình giả lập lại quy trình build vocab y hệt file train.py
    train_src = ["tôi là sinh viên", "máy học rất thú vị"] * 50
    train_tgt = ["i am a student", "machine learning is interesting"] * 50

    # vocab_src = Vocabulary(tokenizer=src_tokenizer, min_freq=1)
    # vocab_tgt = Vocabulary(tokenizer=src_tokenizer, min_freq=1)
    # vocab_src.build_vocabulary(train_src,)
    # vocab_tgt.build_vocabulary(train_tgt,)

    vocab_src = Vocabulary()
    vocab_tgt = Vocabulary()
    vocab_src.build_vocabulary(train_src, lang='vi')
    vocab_tgt.build_vocabulary(train_tgt, lang='en')

    # 3. Load Model Architecture
    model = Transformer(
        src_vocab_size=len(vocab_src),
        tgt_vocab_size=len(vocab_tgt),
        d_model=256,    # Phải khớp config lúc train
        n_layers=3,     # Phải khớp config lúc train
        n_heads=8,
        d_ff=512,
        dropout=0.1,
        src_pad_idx=vocab_src.stoi["<pad>"],
        tgt_pad_idx=vocab_tgt.stoi["<pad>"]
    ).to(device)

    # 4. Load Model Weights
    try:
        model.load_state_dict(torch.load('transformer_model.pt', map_location=device))
        print("Đã load trọng số mô hình thành công!")
    except FileNotFoundError:
        print("Lỗi: Không tìm thấy file 'transformer_model.pt'. Vui lòng chạy train.py trước.")

    # 5. Chuẩn bị tập Test (Dữ liệu chưa từng gặp khi train)
    # Bạn thay thế phần này bằng dữ liệu IWSLT test thật
    test_data = [
        ("tôi là sinh viên", "i am a student"),
        ("máy học rất thú vị", "machine learning is interesting"),
        ("tôi đi học", "i go to school"), # Câu mới
    ]

    # 6. Tính toán
    score = calculate_bleu(test_data,vocab_src,vocab_tgt,model,device)
    # score = calculate_bleu_score(model, test_data, vocab_src, vocab_tgt, device)

    print("\n" + "="*30)
    print(f"KẾT QUẢ CUỐI CÙNG")
    print(f"BLEU Score: {score:.2f}")
    print("="*30)

    # Đánh giá sơ bộ
    if score > 30:
        print("Đánh giá: Mô hình RẤT TỐT (Chất lượng dịch máy thương mại)")
    elif score > 20:
        print("Đánh giá: Mô hình TỐT (Hiểu được ngữ nghĩa, sai ngữ pháp nhẹ)")
    elif score > 10:
        print("Đánh giá: TRUNG BÌNH (Dịch được từ khóa, cấu trúc câu còn lộn xộn)")
    else:
        print("Đánh giá: KÉM (Cần train nhiều hơn hoặc kiểm tra lại dữ liệu)")

Sử dụng thiết bị: cuda
Đã load trọng số mô hình thành công!

KẾT QUẢ CUỐI CÙNG
BLEU Score: 16.23
Đánh giá: TRUNG BÌNH (Dịch được từ khóa, cấu trúc câu còn lộn xộn)
